In [8]:
import ee

# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize(project='ee-zukozaragosa2003')

In [9]:
import pandas as pd
import datetime
import plotly.express as px

In [10]:
# Zonal Statistics using for loop

t1 = datetime.datetime.now() 

firstDate = '2011-07-01'
lastDate = '2011-09-01'

# Calling Image Collection from EE catalogue 
rainfall = ee.ImageCollection('UCSB-CHG/CHIRPS/DAILY')\
        .filter(ee.Filter.date(firstDate, lastDate))
print ('No of Rainfall Images: ', rainfall.size().getInfo())

# Calling Feature Collection from EE Assests
table2 = ee.FeatureCollection("users/sukkiisukant/bundelkhand_districts")

print ('No of Feature: ', table2.size().getInfo())

dates = pd.date_range(start=firstDate,end=lastDate, closed='left')
df = pd.DataFrame(list(zip(dates)),columns =['Dates'])

def setProperty(image):
    dict = image.reduceRegion(ee.Reducer.mean(), geom, 5000)
    return image.set(dict)

listOffeat = table2.toList(table2.size());

collectionSize = listOffeat.size().getInfo()
for i in range(collectionSize):
    aoi = ee.Feature(listOffeat.get(i))
    geom = aoi.geometry()
    name = str(aoi.get('NAME_2').getInfo())
    rainMean = rainfall.map(setProperty)
    rain_list = (rainMean.aggregate_array('precipitation').getInfo())
    df[name] = rain_list 

df = df.set_index("Dates")
print(f"Processing time: {datetime.datetime.now()-t1}")

# Download output in CSV format
df.to_csv("Zonal_sts_using_for_loop.csv")

df

No of Rainfall Images:  62
No of Feature:  13


TypeError: DatetimeArray._generate_range() got an unexpected keyword argument 'closed'

In [ ]:
#  Plotting the Data
fig = px.line(df)
fig.show()